In [ ]:
import loader
import evaluator
import numpy as np
import pandas as pd
from types import SimpleNamespace
from online.fuse_parallel_results import fuse_parallel_results

qrels_path = "C:/Users/USER/Desktop/lifestyle/dev/qrels.tsv"
queries_path = "C:/Users/USER/Desktop/lifestyle/dev/questions.search.tsv"

dataset_name = "lifestyle"
import warnings

warnings.filterwarnings("ignore")
qrels = loader.load_qrels(qrels_path)
queries = loader.load_queries(queries_path)
tf_idf_pipeline = loader.tf_idf_pipeline(dataset_name)
bert_sentence_pipeline = loader.bert_sentence_pipeline(dataset_name)
hybrid_pipeline = SimpleNamespace(
    tfidf_pipeline=tf_idf_pipeline,
    bert_sentence_pipeline=bert_sentence_pipeline
)


def evaluate_pipeline(pipeline, queries, qrels, run_fn, top_k=100):
    precisions, recalls, average_precisions, reciprocal_ranks = [], [], [], []

    for qid, query_text in queries.items():
        relevant_docs = qrels.get(qid, [])
        retrieved_docs, _ = run_fn(pipeline, query_text, top_k)

        precisions.append(evaluator.precision_at_k(relevant_docs, retrieved_docs, k=10))
        recalls.append(evaluator.recall_at_k(relevant_docs, retrieved_docs, k=10))
        average_precisions.append(evaluator.average_precision(relevant_docs, retrieved_docs))
        reciprocal_ranks.append(evaluator.reciprocal_rank(relevant_docs, retrieved_docs))

    return {
        "Precision@10": np.mean(precisions),
        "Recall@10": np.mean(recalls),
        "MAP": np.mean(average_precisions),
        "MRR": np.mean(reciprocal_ranks),
    }


def run_tf_idf(pipeline, query_text, top_k):
    return pipeline.run_with_inverted_index(query_text, top_k)


def run_bert(pipeline, query_text, top_k):
    return pipeline.run_with_flat_ip_index(query_text, top_k)


def run_hybrid_serial(pipeline, query_text, top_k):
    candidates_ids, _ = pipeline.tfidf_pipeline.run_with_inverted_index(query_text, 1000)
    reranked_ids, _ = pipeline.bert_sentence_pipeline.run_with_filtered_candidates(query_text, candidates_ids, top_k)
    return reranked_ids, None


def run_hybrid_parallel(pipeline, query_text, top_k, alpha=0.5):
    tfidf_ids, tfidf_scores = pipeline.tfidf_pipeline.run_with_inverted_index(query_text, top_k=100)
    bert_ids, bert_scores = pipeline.bert_sentence_pipeline.run_with_flat_ip_index(query_text, top_k=100)

    fused_ids, _ = fuse_parallel_results(tfidf_ids, tfidf_scores, bert_ids, bert_scores, alpha=alpha, top_k=top_k)
    return fused_ids, None

In [ ]:
tfidf_results = evaluate_pipeline(tf_idf_pipeline, queries, qrels, run_tf_idf)
bert_results = evaluate_pipeline(bert_sentence_pipeline, queries, qrels, run_bert)
hybrid_results = evaluate_pipeline(hybrid_pipeline, queries, qrels, run_hybrid_serial)
hybrid_parallel_results = evaluate_pipeline(
    hybrid_pipeline,
    queries,
    qrels,
    lambda pipeline, qtext, k=10: run_hybrid_parallel(pipeline, qtext, k, alpha=0.5)
)

df = pd.DataFrame(
    [tfidf_results, bert_results, hybrid_results, hybrid_parallel_results],
    index=["TF-IDF", "BERT", "Hybrid-Serial", "Hybrid-Parallel"]
)
print(df.round(4))